## **Primjeri: Multi-AI agenti za rezervaciju hotela**

U današnjem užurbanom svijetu, planiranje poslovnog putovanja uključuje više od samo rezervacije leta i hotelske sobe. Potrebna je razina koordinacije i učinkovitosti koja može biti izazovna za postići. Tu na scenu stupaju Multi-AI agenti, revolucionirajući način na koji upravljamo našim potrebama za putovanjem.

Zamislite da imate tim inteligentnih agenata na raspolaganju, koji zajedno rade kako bi s preciznošću i lakoćom upravljali svakim aspektom vašeg putovanja. Uz našu naprednu AI tehnologiju, stvorili smo specijalizirane agente za usluge rezervacija i organizaciju itinerara, osiguravajući besprijekorno i bezstresno iskustvo putovanja.

Ovo je osnovni scenarij. Kada planiramo poslovno putovanje, trebamo se konzultirati s poslovnim putničkim agentom kako bismo dobili informacije o avionskim kartama, hotelima itd. Kroz AI agente, možemo izgraditi agente za usluge rezervacija i agente za organizaciju itinerara koji će surađivati i unaprijediti razinu inteligencije.


# Inicijalizirajte Azure AI Agent Service i preuzmite informacije o konfiguraciji iz **.env**

### **.env**

Kreirajte .env datoteku

**.env** sadrži connection string za Azure AI Agent Service, model koji koristi AOAI, te odgovarajući Google API Search servis API, ENDPOINT, itd.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Naziv implementacije modela za Azure AI Agent Service"

[**NOTE**] Trebat će vam model s ograničenjem od 100.000 tokena po minuti (Rate Limit) i ograničenjem od 600 zahtjeva po minuti (Request per minute).

  Model možete dobiti u Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Connection string vašeg projekta za Azure AI Agent Service"

  Connection string projekta možete pronaći u pregledu vašeg projekta na AI ​​Foundry Portal ekranu.

- **SERPAPI_SEARCH_API_KEY** = "Vaš SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Vaš SERPAPI Search Endpoint"

Da biste dobili naziv implementacije modela i connection string projekta za Azure AI Agent Service, potrebno je kreirati Azure AI Agent Service. Preporučuje se koristiti [ovaj predložak](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) za direktno kreiranje. (***Napomena:*** Azure AI Agent Service trenutno je dostupan u ograničenim regijama. Preporučuje se da se konzultirate s [ovim linkom](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) kako biste postavili regiju.)

Agent treba pristupiti SERPAPI. Preporučuje se registracija putem [ovog linka](https://serpapi.com/searches). Nakon registracije, možete dobiti jedinstveni API KEY i ENDPOINT.


# Prijava na Azure

Sada se trebate prijaviti na Azure. Otvorite terminal u VScode-u i pokrenite naredbu `az login`.


# Postavljanje

Za pokretanje ove bilježnice, trebat ćete instalirati sljedeće biblioteke. Evo popisa potrebnih biblioteka i odgovarajućih pip install naredbi:

azure-identity: Za autentifikaciju na Azureu.  
requests: Za slanje HTTP zahtjeva.  
semantic-kernel: Za okvir semantičkog kernela (pretpostavlja se da je ovo prilagođena ili specifična biblioteka, možda ćete je trebati instalirati iz određenog izvora ili repozitorija).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Objašnjenje:  
import asyncio: Ovo uvozi modul asyncio, koji pruža podršku za asinkrono programiranje u Pythonu. Omogućuje vam pisanje konkurentnog koda koristeći sintaksu async i await.  
from typing import Annotated: Ovo uvozi tip Annotated iz modula typing. Annotated se koristi za dodavanje metapodataka na tipove u naznakama, što može biti korisno za razne svrhe poput validacije, dokumentacije ili alata.  


In [ ]:
import asyncio,os
from typing import Annotated

# Objašnjenje:
Korištenjem `from dotenv import load_dotenv` i `load_dotenv()`, možete jednostavno upravljati postavkama konfiguracije i osjetljivim informacijama (poput API ključeva i URL-ova baze podataka) u `.env` datoteci, držeći ih odvojenima od izvornog koda i čineći vašu aplikaciju sigurnijom i lakšom za konfiguriranje.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Objašnjenje:

Import Statement: from azure.identity.aio import DefaultAzureCredential: Ova naredba uvozi klasu DefaultAzureCredential iz modula azure.identity.aio. Dio naziva modula "aio" označava da je dizajniran za asinkrone operacije.

Svrha DefaultAzureCredential: Klasa DefaultAzureCredential dio je Azure SDK-a za Python. Ona pruža zadani način autentifikacije s Azure uslugama. Pokušava se autentificirati koristeći više metoda u određenom redoslijedu, kao što su varijable okruženja, upravljani identitet i Azure CLI vjerodajnice.

Asinkrone operacije: Modul "aio" označava da klasa DefaultAzureCredential podržava asinkrone operacije. To znači da je možete koristiti s asyncio za izvođenje zahtjeva za autentifikaciju bez blokiranja.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Objašnjenje:
Uvozi razne module i klase iz paketa semantic_kernel. Evo detaljnog pregleda svakog uvoza:

AgentGroupChat iz semantic_kernel.agents: Ova klasa upravlja funkcionalnostima vezanim za grupni chat AI agenata. AzureAIAgent i AzureAIAgentSettings iz semantic_kernel.agents.azure_ai

AzureAIAgent: Ova klasa se koristi za kreiranje i upravljanje AI agentima koji koriste Azure AI usluge.

AzureAIAgentSettings: Ova klasa se koristi za konfiguriranje postavki za AzureAIAgent. TerminationStrategy iz semantic_kernel.agents.strategies.termination.termination_strategy:

Ova klasa definira strategije za prekidanje izvršavanja AI agenata pod određenim uvjetima. ChatMessageContent iz semantic_kernel.contents.chat_message_content:

Ova klasa se koristi za upravljanje sadržajem chat poruka.
AuthorRole iz semantic_kernel.contents.utils.author_role:

Ova klasa definira različite uloge autora u kontekstu chat poruka.

kernel_function iz semantic_kernel.functions.kernel_function_decorator: Ovaj dekorator se koristi za definiranje kernel funkcija, koje su funkcije koje se mogu izvršavati unutar okvira semantic kernel.
Ovi uvozi postavljaju potrebne komponente za kreiranje i upravljanje AI agentima koji mogu komunicirati u okruženju grupnog chata, moguće za zadatke poput rezervacije hotela ili sličnih aktivnosti.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Objašnjenje:
Zatim uvozimo klasu CodeInterpreterTool iz modula azure.ai.projects.models.

CodeInterpreterTool: Ova klasa je dio Azure AI SDK-a i koristi se za interpretaciju i izvršavanje koda u kontekstu AI projekata. Pruža funkcionalnosti za pokretanje isječaka koda, analizu koda ili integraciju izvršavanja koda unutar AI tijekova rada. 
Ovaj uvoz postavlja potrebnu komponentu za korištenje CodeInterpreterTool-a u vašem projektu, što može biti korisno za zadatke koji uključuju dinamičku interpretaciju i izvršavanje koda.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Objašnjenje: 
Klasa ApprovalTerminationStrategy pruža specifičnu strategiju za prekid rada AI agenta. Agent će prekinuti rad ako posljednja poruka u njegovoj povijesti interakcija sadrži riječ "spremljeno". Ovo može biti korisno u situacijama gdje se zadatak agenta smatra dovršenim nakon što primi potvrdu da je nešto "spremljeno". Definirajte metodu interakcije. Nakon što je plan rezervacije spremljen, može se zaustaviti kada primi signal "spremljeno".


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Objašnjenje:

Ova linija koda inicijalizira objekt AzureAIAgentSettings s zadanim ili unaprijed definiranim postavkama pozivom metode create(). Ovaj objekt postavki (ai_agent_settings) zatim se može koristiti za konfiguriranje i upravljanje instancom AzureAIAgent.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Objašnjenje:
Korištenjem biblioteke requests, možete jednostavno slati HTTP zahtjeve i komunicirati s web servisima u vašem Python kodu.


In [ ]:
import requests

# Objašnjenje:
Ovo je varijabla koja pohranjuje API ključ za pristup SERP (Search Engine Results Page) API usluzi. API ključ je jedinstveni identifikator koji se koristi za autentifikaciju zahtjeva povezanih s vašim računom.

'GOOGLE_SEARCH_API_KEY': Ovo je rezervirani niz. Trebate zamijeniti 'GOOGLE_SEARCH_API_KEY' s vašim stvarnim SERP API ključem.

Svrha: Svrha ovog retka je pohraniti API ključ u varijablu kako bi se mogao koristiti za autentifikaciju zahtjeva prema SERP API usluzi. API ključ je potreban za pristup usluzi i obavljanje pretraga.

Kako dobiti SERP API ključ: Da biste dobili SERP API ključ, slijedite ove opće korake na https://serpapi.com (točni koraci mogu varirati ovisno o specifičnoj SERP API usluzi koju koristite):

Odaberite SERP API uslugu: Postoji nekoliko dostupnih SERP API usluga, poput SerpAPI, Google Custom Search JSON API i drugih. Odaberite onu koja najbolje odgovara vašim potrebama.

Registrirajte se za račun:

Idite na web stranicu odabrane SERP API usluge https://www.serpapi.com i registrirajte se za račun. Možda ćete morati pružiti osnovne informacije i potvrditi svoju email adresu.

Kreirajte API ključ:

Nakon registracije, prijavite se na svoj račun i idite na odjeljak za API ili nadzornu ploču. Potražite opciju za kreiranje ili generiranje novog API ključa.
Kopirajte API ključ:

Kada se API ključ generira, kopirajte ga. Ovaj ključ će se koristiti za autentifikaciju vaših zahtjeva prema SERP API usluzi.
Zamijenite rezervirani niz:

Zamijenite rezervirani niz u vašoj .env datoteci.


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Objašnjenje:
Klasa BookingPlugin pruža metode za rezervaciju hotela i letova koristeći Serpapi.com Google Search API. Konstruira potrebne parametre, šalje API zahtjeve i obrađuje odgovore kako bi vratila relevantne informacije o rezervacijama. API ključ (SERPAPI_SEARCH_API_KEY) i krajnja točka (SERPAPI_SEARCH_ENDPOINT) koriste se za autentifikaciju i slanje zahtjeva Google Search API-ju.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Objašnjenje:
Klasa SavePlugin pruža metodu saving_plan za spremanje planova putovanja koristeći Azure AI usluge. Postavlja Azure vjerodajnice, kreira AI agenta, obrađuje korisničke unose za generiranje i spremanje sadržaja plana putovanja te upravlja operacijama spremanja datoteka i čišćenja. Metoda vraća "Saved" nakon uspješnog završetka.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Objašnjenje:
Ovaj kod postavlja Azure AI agente za upravljanje rezervacijama letova i hotela te spremanje planova putovanja na temelju korisničkih unosa. Koristi Azure vjerodajnice za kreiranje i konfiguriranje agenata, obrađuje korisničke unose putem grupnog chata i osigurava pravilno čišćenje nakon dovršetka zadataka. Agenti koriste specifične dodatke (BookingPlugin i SavePlugin) za obavljanje svojih odgovarajućih zadataka.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije koje proizlaze iz korištenja ovog prijevoda.
